In [2]:
import numpy as np
from scipy.signal import filtfilt, butter
import matplotlib.pyplot as plt
from numpy import cos, pi, arange
from scipy.fftpack import fft
import scipy
import os
import pywt

In [3]:
def butter_lowpass_filter(data, cutoff, SamplingRate, order):
    nyq = 0.5 * SamplingRate
    normal_cutoff = cutoff / nyq
    # get the filter coefficients
    b, a = butter(order, normal_cutoff, btype='low', analog=False, fs=None)
    Filtered_Data = filtfilt(b, a, data)
    return Filtered_Data

def extract_raw_features(signal):
    # Compute statistical features
    mean_value = np.mean(signal)
    variance = np.var(signal)
    skewness = scipy.stats.skew(signal)
    kurtosis = scipy.stats.kurtosis(signal)
    # Compute frequency-domain features using FFT
    fft_result = np.fft.fft(signal)
    spectral_power = np.abs(fft_result) ** 2
    return [mean_value, variance, skewness, kurtosis, spectral_power]

def extract_wavelet_features(signal, wavelet='db4', level=3):
    # Perform wavelet decomposition
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    wavelet_features = []
    # Extract statistical features from wavelet coefficients
    for coeff in coeffs:
        mean_value = np.mean(coeff)
        variance = np.var(coeff)
        energy = np.sum(np.square(coeff))
        entropy = -np.sum(np.square(coeff) * np.log(np.square(coeff)))
        wavelet_features.extend([mean_value, variance, energy, entropy])
    return wavelet_features

In [5]:
input_signals = "3-class"
output_signals = "3-class output"
wavelet = 'db4'  # wavelet type
level = 3  # wavelet decomposition level

for filename in os.listdir(input_signals):
    if filename.endswith(".txt"):
        file_path = os.path.join(input_signals, filename)
        signal_data = np.loadtxt(file_path)

        # Normalize the signal
        mean_value = np.mean(signal_data)
        std_value = np.std(signal_data)
        normalized_signal = (signal_data - mean_value) / std_value

        # Extract features from raw signal
        raw_features = extract_raw_features(normalized_signal)

        #Apply a low-pass filter to remove high-frequency noise
        Filtered_signal = butter_lowpass_filter(normalized_signal, cutoff=2, SamplingRate=176, order=2)

        # Extract features from wavelet coefficients
        wavelet_features = extract_wavelet_features(Filtered_signal, wavelet=wavelet, level=level)

        # Combine raw and wavelet features
        combined_features = raw_features + wavelet_features

        if(filename.startswith("asagi")):
            class_name = "Down.txt" 
        elif(filename.startswith("yukari")):
            class_name = "Up.txt"
        elif(filename.startswith("sol")):
            class_name = "Left.txt"
        elif(filename.startswith("sag")):
            class_name = "Right.txt"
        elif(filename.startswith("kirp")):
            class_name = "Blink.txt"
        else:
            continue

        class_folder = os.path.join(output_signals, class_name)
        os.makedirs(class_folder, exist_ok=True)

        output_file_path = os.path.join(class_folder, filename.split('.')[0] + "_filtered.txt")
        np.savetxt(output_file_path, Filtered_signal)    


[-1.1323390211316337e-16, 0.9999999999999999, -1.5035226007058278, 2.456567139823423, array([2.87650734e-27, 1.34788933e+04, 1.57231472e+03, 6.34074487e+03,
       1.07235213e+03, 3.36307802e+03, 8.92507223e+02, 1.35688944e+03,
       4.99869941e+02, 5.09709085e+02, 1.73619212e+02, 1.60760794e+02,
       6.15666483e+01, 2.69529576e+01, 1.17702760e+02, 1.20345797e+01,
       1.28345624e+02, 3.82559094e+01, 8.29916178e+01, 5.24723787e+01,
       7.41292960e+01, 3.16991359e+01, 1.78650029e+01, 5.16148529e+01,
       5.29651280e-01, 1.78004685e+01, 1.75079777e+01, 1.69398422e+01,
       3.89706660e+01, 1.15507995e+01, 2.25950999e+01, 1.99003141e+01,
       9.14299853e+00, 1.50720467e+00, 5.06761465e+00, 1.94951779e+01,
       8.22547561e+00, 2.83733644e+00, 1.06412013e+01, 6.07561621e+00,
       5.65574909e+00, 4.90518206e+00, 8.58751174e+00, 1.80378712e+00,
       1.17906697e+00, 3.64134017e+00, 5.11755655e+00, 5.92977040e+00,
       4.22513408e-03, 6.66274490e+00, 4.09983359e+00, 1.25944